In [1]:
import pandas as pd
import numpy as np

def to_month_id(year, month):
  return (year - 1996) * 12 + month - 6

n_features = 12 # or 54
lookback = 3
chosen_stocks = ["C"] # "AMZN", "MSFT", "IBM", "INTC", "QCOM", "NVDA", "IBM", "ADBE", "EBAY", "CSCO"

In [2]:
dfFeature = pd.read_csv("./lstm_2004_12.csv")
# dfFeature.loc[dfFeature["return"] == 0, "return"] = 1
dfFeature = dfFeature[dfFeature.QAId.isin(chosen_stocks)]
features = dfFeature.drop(columns=['month_id', 'QAId']).as_matrix()
print(features.shape)

(157, 12)


In [3]:
from sklearn.preprocessing import MinMaxScaler
normalizedFeatures = MinMaxScaler().fit_transform(features) \
                                   .reshape(157, len(chosen_stocks), n_features) \
                                   .reshape(157, len(chosen_stocks) * n_features)
print(normalizedFeatures.shape)

(157, 12)


In [4]:
dfTarget = pd.read_csv("./return_2004_40.csv")
dfTarget["return"] = np.sign(dfTarget["return"])
# dfTarget.loc[dfTarget["return"] == 0, "return"] = 1
dfTarget = dfTarget[dfTarget.QAId.isin(chosen_stocks)]
dfTarget = dfTarget[dfTarget.month_id >= (to_month_id(2004, 1) + lookback)]
targets = MinMaxScaler().fit_transform(dfTarget.drop(columns=['month_id', 'QAId']).as_matrix())
y = targets.reshape(157 - lookback + 1, len(chosen_stocks))
print(y.shape)

(155, 1)


In [5]:
from keras.preprocessing.sequence import TimeseriesGenerator

splittedFeature = normalizedFeatures
print(splittedFeature.shape)

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(157, 12)


In [6]:
from keras.preprocessing.sequence import TimeseriesGenerator

x = np.zeros((157 - lookback + 1, lookback, n_features * len(chosen_stocks)))
y_mock = np.zeros((157, len(chosen_stocks)))

i = 0
for train, test in TimeseriesGenerator(splittedFeature, y_mock, length=lookback, batch_size=1):
  if i > 157 - lookback:
    break
  x[i] = train[0]
  i = i + 1

In [7]:
from sklearn.model_selection import PredefinedSplit

test_months = 26
end_point = 157 - lookback + 1
split_point = 157 - lookback + 1 - test_months

x_train = x[0:split_point - 48].reshape(split_point - 48, lookback, n_features * len(chosen_stocks))
y_train = y[0:split_point - 48].reshape(split_point - 48, len(chosen_stocks))
x_validate = x[split_point - 36:split_point].reshape(36, lookback, n_features * len(chosen_stocks))
y_validate = y[split_point - 36:split_point].reshape(36, len(chosen_stocks))
x_test = x[split_point:end_point].reshape(test_months, lookback, n_features * len(chosen_stocks))
y_test = y[split_point:end_point].reshape(test_months, len(chosen_stocks))

print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
print(x_test.shape)
print(y_test.shape)

# ps = PredefinedSplit(np.append(np.negative(np.ones(60 * 465)), np.zeros(24 * 465))).split(x_train)

# for train_ids, test_ids in ps:
#   print(str(train_ids) + ", " + str(test_ids))

(81, 3, 12)
(81, 1)
(36, 3, 12)
(36, 1)
(26, 3, 12)
(26, 1)


In [8]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten, CuDNNLSTM
from keras.regularizers import l1_l2, l2
from keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

np.random.seed(4103)

def create_model(*param):
    model = Sequential()
    model.add(LSTM(units=256, input_shape=(lookback, n_features * len(chosen_stocks)), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(len(chosen_stocks), activation="relu"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, decay=0.0), metrics=['accuracy'], *param)
    return model

In [9]:
batch_size=30

model = create_model()
train_score = model.fit(x_train, y_train, validation_data=(x_validate, y_validate), batch_size=batch_size, epochs=200)
print(train_score)

Train on 81 samples, validate on 36 samples
Epoch 1/200
81/81 [==============================] - 2s 25ms/step - loss: 7.3195 - acc: 0.4568 - val_loss: 3.9420 - val_acc: 0.3611
Epoch 2/200
81/81 [==============================] - 0s 760us/step - loss: 3.1791 - acc: 0.4568 - val_loss: 3.1221 - val_acc: 0.3611
Epoch 3/200
81/81 [==============================] - 0s 711us/step - loss: 2.5261 - acc: 0.4568 - val_loss: 2.7984 - val_acc: 0.3611
Epoch 4/200
81/81 [==============================] - 0s 709us/step - loss: 2.2972 - acc: 0.4568 - val_loss: 2.5994 - val_acc: 0.3611
Epoch 5/200
81/81 [==============================] - 0s 712us/step - loss: 2.1667 - acc: 0.4568 - val_loss: 2.4572 - val_acc: 0.3611
Epoch 6/200
81/81 [==============================] - 0s 698us/step - loss: 2.0716 - acc: 0.4568 - val_loss: 2.3478 - val_acc: 0.3611
Epoch 7/200
81/81 [==============================] - 0s 681us/step - loss: 1.9676 - acc: 0.4568 - val_loss: 2.2573 - val_acc: 0.3611
Epoch 8/200
81/81 [=======

Epoch 62/200
81/81 [==============================] - 0s 783us/step - loss: 0.6901 - acc: 0.5309 - val_loss: 0.6899 - val_acc: 0.6111
Epoch 63/200
81/81 [==============================] - 0s 808us/step - loss: 0.6904 - acc: 0.4568 - val_loss: 0.6870 - val_acc: 0.6111
Epoch 64/200
81/81 [==============================] - 0s 753us/step - loss: 0.6965 - acc: 0.4691 - val_loss: 0.6849 - val_acc: 0.6111
Epoch 65/200
81/81 [==============================] - 0s 788us/step - loss: 0.6928 - acc: 0.5309 - val_loss: 0.6832 - val_acc: 0.6111
Epoch 66/200
81/81 [==============================] - 0s 758us/step - loss: 0.7131 - acc: 0.4321 - val_loss: 0.6822 - val_acc: 0.5833
Epoch 67/200
81/81 [==============================] - 0s 748us/step - loss: 0.7013 - acc: 0.4321 - val_loss: 0.6812 - val_acc: 0.5833
Epoch 68/200
81/81 [==============================] - 0s 769us/step - loss: 0.6974 - acc: 0.4938 - val_loss: 0.6805 - val_acc: 0.5833
Epoch 69/200
81/81 [==============================] - 0s 724us

Epoch 123/200
81/81 [==============================] - 0s 797us/step - loss: 0.6939 - acc: 0.4938 - val_loss: 0.6685 - val_acc: 0.6667
Epoch 124/200
81/81 [==============================] - 0s 780us/step - loss: 0.6907 - acc: 0.5556 - val_loss: 0.6684 - val_acc: 0.6667
Epoch 125/200
81/81 [==============================] - 0s 761us/step - loss: 0.6788 - acc: 0.5432 - val_loss: 0.6681 - val_acc: 0.6667
Epoch 126/200
81/81 [==============================] - 0s 742us/step - loss: 0.7032 - acc: 0.4691 - val_loss: 0.6680 - val_acc: 0.6667
Epoch 127/200
81/81 [==============================] - 0s 788us/step - loss: 0.6873 - acc: 0.5679 - val_loss: 0.6679 - val_acc: 0.6667
Epoch 128/200
81/81 [==============================] - 0s 879us/step - loss: 0.6809 - acc: 0.6049 - val_loss: 0.6679 - val_acc: 0.6667
Epoch 129/200
81/81 [==============================] - 0s 828us/step - loss: 0.6812 - acc: 0.5926 - val_loss: 0.6681 - val_acc: 0.6667
Epoch 130/200
81/81 [==============================] - 

Epoch 184/200
81/81 [==============================] - 0s 819us/step - loss: 0.6686 - acc: 0.6296 - val_loss: 0.6702 - val_acc: 0.6389
Epoch 185/200
81/81 [==============================] - 0s 788us/step - loss: 0.6691 - acc: 0.5679 - val_loss: 0.6711 - val_acc: 0.6389
Epoch 186/200
81/81 [==============================] - 0s 754us/step - loss: 0.6685 - acc: 0.6173 - val_loss: 0.6717 - val_acc: 0.6389
Epoch 187/200
81/81 [==============================] - 0s 817us/step - loss: 0.6776 - acc: 0.5556 - val_loss: 0.6717 - val_acc: 0.6389
Epoch 188/200
81/81 [==============================] - 0s 858us/step - loss: 0.6700 - acc: 0.6296 - val_loss: 0.6717 - val_acc: 0.6389
Epoch 189/200
81/81 [==============================] - 0s 877us/step - loss: 0.6826 - acc: 0.5556 - val_loss: 0.6713 - val_acc: 0.6389
Epoch 190/200
81/81 [==============================] - 0s 926us/step - loss: 0.6711 - acc: 0.5679 - val_loss: 0.6718 - val_acc: 0.6389
Epoch 191/200
81/81 [==============================] - 

In [10]:
import matplotlib.pyplot as plt

test_score = model.evaluate(x_test, y_test, batch_size=batch_size)
for i, mn in enumerate(model.metrics_names):
  print(mn + ": " + str(test_score[i]))

plt.plot(train_score.history["loss"])
plt.plot(train_score.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Metrics")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

26/26 [==============================] - 0s 192us/step
loss: 0.6556978821754456
acc: 0.6153846383094788


In [11]:
from sklearn.metrics import precision_score, recall_score, classification_report

def threshold_tester(result, target, a, b):
  for t in np.arange(a, b + 0.05, 0.05):
    print("thresholder: " + str(t))
    print(classification_report(target, (result >= t).astype(int)))

In [12]:
result = model.predict(x_test, batch_size=batch_size)
print(result)
threshold_tester(result, y_test, 0.1, 0.9)

[[0.61844635]
 [0.58979595]
 [0.59548956]
 [0.58125585]
 [0.64216036]
 [0.5212378 ]
 [0.64241016]
 [0.59042937]
 [0.56475776]
 [0.57961726]
 [0.5923262 ]
 [0.5218785 ]
 [0.5179794 ]
 [0.48085964]
 [0.5322363 ]
 [0.52420485]
 [0.5176543 ]
 [0.53222585]
 [0.5861822 ]
 [0.54779375]
 [0.58428913]
 [0.5791396 ]
 [0.611796  ]
 [0.6051867 ]
 [0.6154722 ]
 [0.64887685]]
thresholder: 0.1
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         9
        1.0       0.65      1.00      0.79        17

avg / total       0.43      0.65      0.52        26

thresholder: 0.15000000000000002
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         9
        1.0       0.65      1.00      0.79        17

avg / total       0.43      0.65      0.52        26

thresholder: 0.20000000000000004
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         9
        1.0    

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
import math

buy_list = []
sell_list = []

for j, stock in enumerate(chosen_stocks):
  print(stock)
  sorted_result = sorted(map(lambda x: x[j], result))
  midpt = (sorted_result[-2] + sorted_result[1]) / 2
  upper_threshold = midpt * 1.05
  lower_threshold = midpt * 0.95
  
  print("Target\tPredict\tConsequence")
  for i, r in enumerate(result):
    prediction = r[j].item()
    target = y_test[i][j].item()
    buy_or_sell = 1 if prediction > upper_threshold else (-1 if prediction < lower_threshold else 0)
    if prediction > upper_threshold:
      buy_list.append({'month_id': i + 223, 'QAId': stock})
    if prediction < lower_threshold:
      sell_list.append({'month_id': i + 223, 'QAId': stock})
    
    to_print = str(target) + "\t" + str(buy_or_sell)
    if (buy_or_sell == -1 and target == 0) or (buy_or_sell == 1 and target == 1):
      print(to_print + "\tGain")
    elif (buy_or_sell == -1 and target == 1) or (buy_or_sell == 1 and target == 0):
      print(to_print + "\tLoss")
    else:
      print(to_print + "\tNothing")
  print("\n")

print(buy_list)
print(sell_list)

C
Target	Predict	Consequence
1.0	1	Gain
0.0	0	Nothing
1.0	0	Nothing
1.0	0	Nothing
1.0	1	Gain
1.0	-1	Loss
0.0	1	Loss
0.0	0	Nothing
1.0	0	Nothing
1.0	0	Nothing
0.0	0	Nothing
0.0	-1	Gain
0.0	-1	Gain
1.0	-1	Loss
1.0	-1	Loss
1.0	-1	Loss
0.0	-1	Gain
1.0	-1	Loss
1.0	0	Nothing
0.0	-1	Gain
1.0	0	Nothing
1.0	0	Nothing
1.0	1	Gain
0.0	0	Nothing
1.0	1	Gain
1.0	1	Gain


[{'month_id': 223, 'QAId': 'C'}, {'month_id': 227, 'QAId': 'C'}, {'month_id': 229, 'QAId': 'C'}, {'month_id': 245, 'QAId': 'C'}, {'month_id': 247, 'QAId': 'C'}, {'month_id': 248, 'QAId': 'C'}]
[{'month_id': 228, 'QAId': 'C'}, {'month_id': 234, 'QAId': 'C'}, {'month_id': 235, 'QAId': 'C'}, {'month_id': 236, 'QAId': 'C'}, {'month_id': 237, 'QAId': 'C'}, {'month_id': 238, 'QAId': 'C'}, {'month_id': 239, 'QAId': 'C'}, {'month_id': 240, 'QAId': 'C'}, {'month_id': 242, 'QAId': 'C'}]


In [14]:
dfBuy = pd.DataFrame(buy_list, columns=["month_id", "QAId"])
dfSell = pd.DataFrame(sell_list, columns=["month_id", "QAId"])
dfBuy.head(5)

,month_id,QAId
0,223,C
1,227,C
2,229,C
3,245,C
4,247,C


In [15]:
dfSell.head(5)

,month_id,QAId
0,228,C
1,234,C
2,235,C
3,236,C
4,237,C


In [16]:
filename_base = "_".join([stock.lower() for stock in chosen_stocks])
# filename_model = "./" + filename_base + "_model.h5"
# filename_weights = "./" + filename_base + "_weights.h5"
filename_output_buy = "./" + filename_base + "_output_buy.h5"
filename_output_sell = "./" + filename_base + "_output_sell.h5"

# model.save(filename_model)
# model.save_weights(filename_weights)
dfBuy.to_csv(filename_output_buy, index=False)
dfSell.to_csv(filename_output_sell, index=False)

# files.download(filename_model)
# files.download(filename_weights)
# files.download(filename_output_buy)
# files.download(filename_output_sell)